# Exercise 1 MLP with PyTorch for CIFAR10

Based on the notebooks provided and discussed during the lecture, set up a notebook for the configuration and training of a MLP on the CIFAR10 images.

[These images](https://www.cs.toronto.edu/~kriz/cifar.html) of size 32x32 pixel show 10 different object categories

Your implementation should provide the following features : 

- Download of the CIFAR10 images form an appropriate server and local storage for further usage.
- Configuration of a custom Dataset (c.f. chapter 6.2.2 in the lecture notes) that will provide the CIFAR10 data to the DataLoader class during training.
- Set up of a SummaryWriter for TensorBoard (c.f. chapter 6.2.3 in the lecture notes) and output of a set of CIFAR10 images (as grid like the figures above) to TensorBoard.
- Set up of a MLP with confiigurable number of hidden layers that takes CIFAR10 (colour) images as input.
- Preparation of the SummaryWriter for continuous output of training and validation loss/error to TensorBoard during the training process.

In [1]:
# ~~~ Imports ~~~
import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks import ModelCheckpoint
from torch import nn, optim
from torch.utils.data import DataLoader
from torchvision import transforms, datasets


In [2]:
# ~~~ PyTorch Lightning Data Module ~~~
class CIFAR10DataModule(pl.LightningDataModule):
    def __init__(self, batch_size=128):
        super().__init__()
        self.batch_size = batch_size

    def setup(self, stage=None):
        # Setup data only once per GPU
        self.train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
        self.test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

    def train_dataloader(self):
        return DataLoader(self.train_dataset, batch_size=self.batch_size, shuffle=True, num_workers=20)
    
    def val_dataloader(self):
        return DataLoader(self.test_dataset, batch_size=self.batch_size, shuffle=False, num_workers=20)

    def test_dataloader(self):
        return DataLoader(self.test_dataset, batch_size=self.batch_size, num_workers=20)

In [3]:
# ~~~ Transforms ~~~
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

In [4]:
# ~~~ Builder Functions ~~~
def conv_block(in_channels, out_channels, kernel_size=3, stride=1, padding=1):
    return nn.Sequential(
        nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, stride=stride, padding=padding),
        nn.ReLU(),
        nn.BatchNorm2d(out_channels)
    )

def make_layers(in_channels, n_blocks=4, n_convs=3, out_channels=64, kernel_size=3, pool_size=2, dropout_rate=0.2):
    layers = []
    for _ in range(n_blocks):
        for _ in range(n_convs):
            layers.append(conv_block(in_channels, out_channels, kernel_size))
            in_channels = out_channels  # Output becomes input for the next layer
        layers.append(nn.MaxPool2d(kernel_size=pool_size, stride=pool_size))
        layers.append(nn.Dropout(dropout_rate))
    return nn.Sequential(*layers)

In [5]:

# ~~~ Convolutional Neural Network ~~~
class VGGNetPL(pl.LightningModule):
    def __init__(self, in_channels=3, out_channels=64, n_blocks=4, n_convs=3, dropout_rate=0.2):
        super(VGGNetPL, self).__init__()
        self.features = make_layers(in_channels, n_blocks, n_convs, out_channels, dropout_rate=dropout_rate)
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(out_channels * 2 * 2, 256),  # The 2*2 is derived assuming input is 32x32 and goes through four pooling layers
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, 10)
        )
        self.criterion = nn.CrossEntropyLoss()

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

    def training_step(self, batch, batch_idx):
        images, labels = batch
        outputs = self(images)
        loss = self.criterion(outputs, labels)
        self.log('train_loss', loss)
        return loss

    def validation_step(self, batch, batch_idx):
        images, labels = batch
        outputs = self(images)
        loss = self.criterion(outputs, labels)
        self.log('val_loss', loss)

    def test_step(self, batch, batch_idx):
        images, labels = batch
        outputs = self(images)
        loss = self.criterion(outputs, labels)
        self.log('test_loss', loss)
        return {'test_loss': loss}


    def configure_optimizers(self, learning_rate=0.001):
        return optim.Adam(self.parameters(), lr=learning_rate)

In [6]:
# ~~~ Parameters ~~~
batch_size = 512
num_epochs = 10
learning_rate = 0.001

In [7]:
# ~~~ Setup training ~~~
data_module = CIFAR10DataModule(batch_size=batch_size)
model = VGGNetPL()
model.configure_optimizers(learning_rate)
logger = TensorBoardLogger('tb_logs', name='vgg_cifar10')

trainer = pl.Trainer(max_epochs=num_epochs, logger=logger, callbacks=[ModelCheckpoint(monitor="val_loss")])
trainer.fit(model, data_module)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Files already downloaded and verified
Files already downloaded and verified


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name       | Type             | Params
------------------------------------------------
0 | features   | Sequential       | 409 K 
1 | classifier | Sequential       | 68.4 K
2 | criterion  | CrossEntropyLoss | 0     
------------------------------------------------
477 K     Trainable params
0         Non-trainable params
477 K     Total params
1.912     Total estimated model params size (MB)


Epoch 9: 100%|██████████| 98/98 [00:08<00:00, 11.36it/s, v_num=5]          

`Trainer.fit` stopped: `max_epochs=10` reached.


Epoch 9: 100%|██████████| 98/98 [00:08<00:00, 11.33it/s, v_num=5]


In [8]:
# ~~~ Setup Evaluation ~~~
trainer.test(model, datamodule=data_module)

Files already downloaded and verified
Files already downloaded and verified


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|██████████| 20/20 [00:00<00:00, 44.85it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_loss           0.5163008570671082
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_loss': 0.5163008570671082}]